### 6.2.1 변동성 돌파 전략
1) 가격 변동폭 계산: 투자하려는 가상화폐의 전일 고가(high)에서 전일 저가(low)를 빼서 가상화폐의 가격 변동폭을 구한다.  
2) 매수 기준: 당일 시간에서 (변동폭 * 0.5) 이상 상승하면 해당 가격에 바로 매수한다  
3) 매도 기준: 당일 종가에서 매도한다.

### 6.2.2 1단계: 주기적으로 현재가 얻어오기

In [1]:
import pyupbit
import time

In [2]:
while True:
    price = pyupbit.get_current_price('BTC-ETH')
    print(price)
    time.sleep(0.2)

0.054
0.054
0.054
0.054
0.054
0.054
0.054
0.054
0.054
0.054
0.054


KeyboardInterrupt: 

### 6.2.3 2단계: 목표가 계산하기

In [3]:
# get_ohlcv(): 시가/고가/저가/종가/거래량 일봉 데이터를 데이터프레임 객체로 반환
df = pyupbit.get_ohlcv('BTC-ETH')
print(df.tail())

                         open      high       low     close     volume  \
2023-11-03 09:00:00  0.051905  0.053141  0.051460  0.052513  17.839899   
2023-11-04 09:00:00  0.052443  0.053151  0.052443  0.052745  31.925513   
2023-11-05 09:00:00  0.052745  0.054200  0.051950  0.054169  37.295196   
2023-11-06 09:00:00  0.053777  0.054572  0.053669  0.054572   7.690913   
2023-11-07 09:00:00  0.054411  0.054478  0.054000  0.054000   5.198513   

                        value  
2023-11-03 09:00:00  0.929879  
2023-11-04 09:00:00  1.680360  
2023-11-05 09:00:00  1.987149  
2023-11-06 09:00:00  0.415187  
2023-11-07 09:00:00  0.280954  


In [4]:
yesterday = df.iloc[-2]

In [6]:
today_open = yesterday['close']
yesterday_high = yesterday['high']
yesterday_low = yesterday['low']
target = today_open + (yesterday_high - yesterday_low) * 0.5
target

0.05502357000000001

In [7]:
def get_target_price(ticker):
    df = pyupbit.get_ohlcv(ticker)
    yesterday = df.iloc[-2]
    
    today_open = yesterday['close']
    yesterday_high = yesterday['high']
    yesterday_low = yesterday['low']
    target = today_open + (yesterday_high - yesterday_low) * 0.5
    return target

### 6.2.4 3단계: 자정에 목표가 갱신하기

In [8]:
import datetime

In [9]:
dt = datetime.datetime(2018, 12, 1)
print(dt)
print(dt.year, dt.month, dt.day)

2018-12-01 00:00:00
2018 12 1


In [10]:
now = datetime.datetime.now()
print(now)

2023-11-07 23:12:29.917087


In [11]:
print(now == dt)
print(now > dt)

False
True


In [12]:
mid = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(1) # timedelta(n): n일의 시간
print(mid)

2023-11-08 00:00:00


In [13]:
import time
import datetime

now = datetime.datetime.now()
mid = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(1)

# 프로그램에서 정확하기 1초 단위로 시각을 비교할 수 없음
while True:
    now = datetime.datetime.now()
    if now == mid:
        print('정각입니다')
        now = datetime.datetime.now()
        mid = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(1)
        
    print(now, 'vs', mid)
    time.sleep(1)

2023-11-07 23:25:25.785711 vs 2023-11-08 00:00:00
2023-11-07 23:25:26.800190 vs 2023-11-08 00:00:00
2023-11-07 23:25:27.811531 vs 2023-11-08 00:00:00
2023-11-07 23:25:28.826270 vs 2023-11-08 00:00:00
2023-11-07 23:25:29.839084 vs 2023-11-08 00:00:00
2023-11-07 23:25:30.855266 vs 2023-11-08 00:00:00
2023-11-07 23:25:31.867759 vs 2023-11-08 00:00:00
2023-11-07 23:25:32.881380 vs 2023-11-08 00:00:00
2023-11-07 23:25:33.893100 vs 2023-11-08 00:00:00


KeyboardInterrupt: 

In [ ]:
import time
import datetime

now = datetime.datetime.now()
mid = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(1)

# 프로그램에서 정확하기 1초 단위로 시각을 비교할 수 없음
while True:
    now = datetime.datetime.now()
    if mid < now < mid + datetime.timedelta(seconds=10): # seconds=n: n초를 더함
        print('정각입니다')
        mid = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(1)
        
    time.sleep(1)

### 6.2.5 4단계: 매수 시도

In [ ]:
import time
import datetime

now = datetime.datetime.now()
mid = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(1)

# 프로그램에서 정확하기 1초 단위로 시각을 비교할 수 없음
while True:
    now = datetime.datetime.now()
    if mid < now < mid + datetime.timedelta(seconds=10): # seconds=n: n초를 더함
        print('정각입니다')
        mid = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(1)
        
        
    current_price = pyupbit.get_current_price('BTC-ETH')
    if current_price > target_price: # 현재가가 목표가보다 큰 경우 실행
        krw = upbit.get_balance('BTC-ETH')[2] # 잔고 조회 > 보유 중인 원화 얻어옴
        orderbook = pyupbit.get_orderbook('BTC-ETH') # 호가창 및 최우선 매도 호가 조회
        sell_price = orderbook['asks'][0]['price']
        unit = krw / float(sell_price) # 구매 가능 수량 계산
        upbit.buy_market_order('BTC-ETH', unit) # 시장가 주문 매수
    
    time.sleep(1)

In [14]:
def buy_crypto_currency(ticker):
        krw = upbit.get_balance(ticker)[2] # 잔고 조회 > 보유 중인 원화 얻어옴
        orderbook = pyupbit.get_orderbook(ticker) # 호가창 및 최우선 매도 호가 조회
        sell_price = orderbook['asks'][0]['price']
        unit = krw / float(sell_price) # 구매 가능 수량 계산
        upbit.buy_market_order(ticker, unit) # 시장가 주문 매수

### 6.2.6 5단계: 매도 시도
* 변동성 돌파 전략 5단계: 보유 중인 비트코인을 다음 날 시초가에 전량 매도

In [ ]:
while True:
    now = datetime.datetime.now()
    if mid < now < mid + datetime.timedelta(seconds=10): # seconds=n: n초를 더함
        target_price = get_target_price('BTC-ETH')
        now = datetime.datetime.now()
        mid = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(1)
        sell_crypto_currency('BTC-ETH')

In [15]:
def sell_crypto_currency(ticker):
    unit = upbit.get_balance(ticker)[0]
    upbit.sell_market_order(ticker, unit)

In [ ]:
while True:
    now = datetime.datetime.now()
    if mid < now < mid + datetime.timedelta(seconds=10): # seconds=n: n초를 더함
        target_price = get_target_price('BTC-ETH')
        now = datetime.datetime.now()
        mid = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(1)
        sell_crypto_currency('BTC-ETH')
        
    current_price = pyupbit.get_current_price('BTC-ETH')
    if current_price > target_price:
        buy_crypto_currency('BTC-ETH')
        
    time.sleep(1)

### 6.2.7 6단계: 보안 및 예외처리

In [17]:
with open('upbit.txt') as f:
    lines = f.readlines()
    key = lines[0].strip()
    secret = lines[1].strip()
    upbit = pyupbit.Upbit(key, secret)

In [ ]:
while True:
    try:
        now = datetime.datetime.now()
        if mid < now < mid + datetime.timedelta(seconds=10): # seconds=n: n초를 더함
            target_price = get_target_price('BTC-ETH')
            now = datetime.datetime.now()
            mid = datetime.datetime(now.year, now.month, now.day) + datetime.timedelta(1)
            sell_crypto_currency('BTC-ETH')

        current_price = pyupbit.get_current_price('BTC-ETH')
        if current_price > target_price:
            buy_crypto_currency('BTC-ETH')
        
    except:
        print('에러 발생')
        
    time.sleep(1)